In [1]:
import spacy

#import model as model_config
#from data_utils import load as load_data, extract_features
import numpy as np


%load_ext autoreload
%autoreload 2

import re


import spacy
import pickle
import pandas as pd
import numpy as np
import time

from sklearn.model_selection import ShuffleSplit
from sklearn.preprocessing import LabelEncoder


twitter_data_neg_small = './twitter-datasets/train_neg.txt'
twitter_data_pos_small = './twitter-datasets/train_pos.txt'
twitter_data_small     = './twitter-datasets/train_small.txt'

twitter_data_neg_full  = './twitter-datasets/train_neg_full.txt'
twitter_data_pos_full  = './twitter-datasets/train_pos_full.txt'
twitter_data_full      = './twitter-datasets/train_full.txt'
nlp = spacy.load('en_core_web_lg')

In [2]:
# Use dictionary from http://luululu.com/tweet/typo-corpus-r1.txt
# http://people.eng.unimelb.edu.au/tbaldwin/etc/emnlp2012-lexnorm.tgz
# to handle abbreviations, mistakes...etc. (IN )

# LULU-CORPUS
# (1) INSERT (IN): a character is added to the original word.
# (2) REMOVE (RM): a character is removed from the original word.
# (3) REPLACE1 (R1): the order of character is different from the original word (the number of differences is one).
# (4) REPLACE2 (R2): a character is different from the original word

final_corpus = {}

def corpusReplace(corpus):
    for word in corpus:
        word = word.decode('utf8')
        word = word.split()
        final_corpus[word[0]] = word[1]    

corpus_lulu = open('corpus/lulu-corpus.txt', 'rb')
corpusReplace(corpus_lulu)
corpus_lulu.close()

corpus_emnlp = open('corpus/emnlp-corpus.txt', 'rb')
corpusReplace(corpus_emnlp)
corpus_emnlp.close()

def applyCorpus(tweet):
    new_tweet = ''
    for w in tweet.split(' '):
        if w in final_corpus.keys():
            #Replace with correct value
            new_word = final_corpus[w]
            new_tweet = new_tweet + ' ' + new_word
        else:
            new_tweet = new_tweet + ' ' + w
    return new_tweet

#raw_data_train['text'] = raw_data_train.text.apply(applyCorpus)   
         
def cleanTweet(tweet):
    tweet = re.sub('<url>','',tweet)
    tweet = re.sub('<user>','',tweet)
    tweet = re.sub(r'\$\w*', '', tweet)
    tweet = tweet.lower()
    tweet = re.sub(r'https?:\/\/.*\/\w*', '', tweet)
    tweet = re.sub(r'#\w*', '', tweet) #hashtag
    tweet = re.sub(r'[' + punctuation.replace('@', '') + ']+', ' ', tweet) # puntuaction
    tweet = re.sub(r'\s\s+', ' ', tweet)
    tweet = tweet.lstrip(' ') 
    tweet = ''.join(c for c in tweet if c <= '\uFFFF') 
    return tweet

#raw_data_train['text'] = raw_data_train.text.apply(cleanTweet)

def extract_features(docs, max_length):
    docs = list(docs)
    X = np.zeros((len(docs), max_length), dtype='int32')
    for i, doc in enumerate(docs):
        j = 0
        for token in doc:
            if token.has_vector and not token.is_punct and not token.is_space:
                X[i, j] = token.rank + 1
                j += 1
                if j >= max_length:
                    break
    return X

def load_twitter_data_small(from_cache=False):
    cached_data_path = twitter_data_small + '.cached.pkl'

    if from_cache:
        print('Loading data from cache...')
        with open(cached_data_path, 'rb') as f:
            return pickle.load(f)

    max_length = 100

    print('Loading and preparing data...')
    raw_data_neg = pd.read_csv(twitter_data_neg_small, header=None, sep="\n", encoding='latin1', names=['text'],
                               error_bad_lines=False, warn_bad_lines=False, quoting=csv.QUOTE_NONE).drop_duplicates()
    raw_data_neg['text'] = raw_data_neg['text'].apply(cleanTweet)
    raw_data_neg['text'] = raw_data_neg['text'].apply(applyCorpus)
    raw_data_neg['label'] = 0
    raw_data_neg = raw_data_neg

    raw_data_pos = pd.read_csv(twitter_data_pos_small, header=None, sep="\n", encoding='latin1', names=['text'],
                               error_bad_lines=False, warn_bad_lines=False, quoting=csv.QUOTE_NONE).drop_duplicates()
    raw_data_pos['text'] = raw_data_pos['text'].apply(cleanTweet)
    raw_data_pos['text'] = raw_data_pos['text'].apply(applyCorpus)
    raw_data_pos['label'] = 1
    raw_data_pos = raw_data_pos
    
    raw_data = pd.concat([raw_data_neg, raw_data_pos], ignore_index=True)
#     raw_data = raw_data[:10000]
    
    #print(raw_data)

    # Parse tweet texts
    docs = list(nlp.pipe(raw_data['text'], batch_size=1000, n_threads=8))
    print(max([len(x) for x in docs])) 
    
    y = raw_data['label'].values
    
    # Pull the raw_data into vectors
    X = extract_features(docs, max_length=max_length)
    
    # Split into train and test sets
    rs = ShuffleSplit(n_splits=2, random_state=42, test_size=0.2)
    train_indices, test_indices = next(rs.split(X))
    
    X_train = X[train_indices]
    y_train = y[train_indices]
    X_test = X[test_indices]
    y_test = y[test_indices]
    
    docs = np.array(docs, dtype=object)
    docs_train = docs[train_indices]
    docs_test = docs[test_indices]
    
    numeric_data = X_train, y_train, X_test, y_test
    raw_data = docs_train, docs_test

    with open(cached_data_path, 'wb') as f:
        pickle.dump((numeric_data, raw_data), f)
    
    return numeric_data, raw_data

def load_twitter_data_full(from_cache=False):
    cached_data_path = twitter_data_full + '.cached.pkl'

    if from_cache:
        print('Loading data from cache...')
        with open(cached_data_path, 'rb') as f:
            return pickle.load(f)

    max_length = 100

    print('Loading and preparing data...')
    raw_data_neg = pd.read_csv(twitter_data_neg_full, header=None, sep="\n", encoding='latin1', names=['text'],
                               error_bad_lines=False, warn_bad_lines=False, quoting=csv.QUOTE_NONE).drop_duplicates()
    raw_data_neg['text'] = raw_data_neg['text'].apply(cleanTweet)
    raw_data_neg['text'] = raw_data_neg['text'].apply(applyCorpus)
    raw_data_neg['label'] = 0
    raw_data_neg = raw_data_neg

    raw_data_pos = pd.read_csv(twitter_data_pos_full, header=None, sep="\n", encoding='latin1', names=['text'],
                               error_bad_lines=False, warn_bad_lines=False, quoting=csv.QUOTE_NONE).drop_duplicates()
    raw_data_pos['text'] = raw_data_pos['text'].apply(cleanTweet)
    raw_data_pos['text'] = raw_data_pos['text'].apply(applyCorpus)
    raw_data_pos['label'] = 1
    raw_data_pos = raw_data_pos
    
    raw_data = pd.concat([raw_data_neg, raw_data_pos], ignore_index=True)
#     raw_data = raw_data[:10000]
    
    #print(raw_data)

    # Parse tweet texts
    docs = list(nlp.pipe(raw_data['text'], batch_size=5000, n_threads=8))
    print(max([len(x) for x in docs])) 
    
    y = raw_data['label'].values
    
    # Pull the raw_data into vectors
    X = extract_features(docs, max_length=max_length)
    
    # Split into train and test sets
    rs = ShuffleSplit(n_splits=2, random_state=42, test_size=0.2)
    train_indices, test_indices = next(rs.split(X))
    
    X_train = X[train_indices]
    y_train = y[train_indices]
    X_test = X[test_indices]
    y_test = y[test_indices]
    
    docs = np.array(docs, dtype=object)
    docs_train = docs[train_indices]
    docs_test = docs[test_indices]
    
    numeric_data = X_train, y_train, X_test, y_test
    raw_data = docs_train, docs_test

    with open(cached_data_path, 'wb') as f:
        pickle.dump((numeric_data, raw_data), f)
    
    return numeric_data, raw_data

def load(data_name, *args, **kwargs):
    load_fn_map = {
        'twitter_data_small': load_twitter_data_small,
        'twitter_data_full': load_twitter_data_full,
    }
    return load_fn_map[data_name](*args, **kwargs)

In [3]:
# Load Twitter data
(X_train, y_train, X_test, y_test), (docs_train, docs_test) = load('twitter_data_small', from_cache=True)

Loading data from cache...


In [4]:
print('X_train:', X_train.shape)
print('y_train:', y_train.shape)
print('X_test:', X_test.shape)
print('y_test:', y_test.shape)

X_train: (145056, 100)
y_train: (145056,)
X_test: (36265, 100)
y_test: (36265,)


In [5]:
import spacy
import keras
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Flatten, Conv1D, MaxPooling1D, GlobalAveragePooling1D, LSTM, merge
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.constraints import maxnorm
from keras.regularizers import l2
from keras import backend as K

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [6]:
def get_embeddings(vocab):
    max_rank = max(lex.rank+1 for lex in vocab if lex.has_vector)
    vectors = np.ndarray((max_rank+1, vocab.vectors_length), dtype='float32')
    for lex in vocab:
        if lex.has_vector:
            vectors[lex.rank + 1] = lex.vector
    return vectors


vocab_nlp = spacy.load('en_core_web_lg', parser=False, tagger=False, entity=False)
print('Preparing embeddings...')
embeddings = get_embeddings(vocab_nlp.vocab)

Preparing embeddings...


In [9]:
def build_model_twitter(max_length=100,
                nb_filters=64,
                kernel_size=3,
                pool_size=2,
                regularization=0.01,
                weight_constraint=2.,
                dropout_prob=0.4,
                clear_session=True):
    if clear_session:
        K.clear_session()

    model = Sequential()
    model.add(Embedding(
        embeddings.shape[0],
        embeddings.shape[1],
        input_length=max_length,
        trainable=False,
        weights=[embeddings]))

    model.add(Conv1D(nb_filters, kernel_size, activation='relu'))
    model.add(Conv1D(nb_filters, kernel_size, activation='relu'))
    model.add(MaxPooling1D(pool_size))

    model.add(Dropout(dropout_prob))

    model.add(Conv1D(nb_filters * 2, kernel_size, activation='relu'))
    model.add(Conv1D(nb_filters * 2, kernel_size, activation='relu'))
    model.add(MaxPooling1D(pool_size))

    model.add(Dropout(dropout_prob))

    model.add(GlobalAveragePooling1D())
    model.add(Dense(1,
        kernel_regularizer=l2(regularization),
        kernel_constraint=maxnorm(weight_constraint),
        activation='sigmoid'))

    model.compile(
        loss='binary_crossentropy',
        optimizer='adam',
        metrics=['accuracy'])

    return model

def build_model_twitter2(max_length=100,
                nb_filters=64,
                kernel_size=3,
                pool_size=2,
                regularization=0.01,
                weight_constraint=2.,
                dropout_prob=0.4,
                clear_session=True):
    if clear_session:
        K.clear_session()

    model = Sequential()
    model.add(Embedding(
        embeddings.shape[0],
        embeddings.shape[1],
        input_length=max_length,
        trainable=False,
        weights=[embeddings]))

    model.add(Conv1D(nb_filters, kernel_size, activation='relu'))
    model.add(MaxPooling1D(pool_size))
    model.add(Dropout(dropout_prob))

    model.add(Conv1D(nb_filters * 2, kernel_size, activation='relu'))
    model.add(MaxPooling1D(pool_size))
    model.add(Dropout(dropout_prob))
    
    model.add(Conv1D(nb_filters * 4, kernel_size, activation='relu'))
    model.add(MaxPooling1D(pool_size))
    model.add(Dropout(dropout_prob))

    model.add(GlobalAveragePooling1D())
    model.add(Dense(1,
        kernel_regularizer=l2(regularization),
        kernel_constraint=maxnorm(weight_constraint),
        activation='sigmoid'))

    model.compile(
        loss='binary_crossentropy',
        optimizer='adam',
        metrics=['accuracy'])

    return model

def build_model_twitter3(max_length=100,
                nb_filters=64,
                kernel_size=3,
                pool_size=2,
                regularization=0.01,
                weight_constraint=2.,
                dropout_prob=0.4,
                clear_session=True):
    if clear_session:
        K.clear_session()

    model = Sequential()
    model.add(Embedding(
        embeddings.shape[0],
        embeddings.shape[1],
        input_length=max_length,
        trainable=False,
        weights=[embeddings]))

    model.add(Conv1D(nb_filters, kernel_size))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size))
    model.add(Activation('relu'))
    model.add(Dropout(dropout_prob))

    model.add(Conv1D(nb_filters * 2, kernel_size))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size))
    model.add(Activation('relu'))
    model.add(Dropout(dropout_prob))

    model.add(GlobalAveragePooling1D())
    model.add(Dense(1,
        kernel_regularizer=l2(regularization),
        kernel_constraint=maxnorm(weight_constraint),
        activation='sigmoid'))

    model.compile(
        loss='binary_crossentropy',
        optimizer='adam',
        metrics=['accuracy'])

    return model

In [10]:
#Model 1
np.random.seed(42)

epochs = 12
batch_size = 128


# Take a look at the shapes
print('X_train:', X_train.shape)
print('y_train:', y_train.shape)
print('X_test:', X_test.shape)
print('y_test:', y_test.shape)

tb_callback = keras.callbacks.TensorBoard(
        histogram_freq=0, write_graph=True)

model = build_model_twitter()
model.fit(X_train, y_train,
          validation_data=(X_test, y_test),
          batch_size=batch_size, epochs=epochs,
          callbacks=[tb_callback])

X_train: (145056, 100)
y_train: (145056,)
X_test: (36265, 100)
y_test: (36265,)
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Train on 145056 samples, validate on 36265 samples
Epoch 1/12
145056/145056 [==============================] - 107s - loss: 0.5181 - acc: 0.7373 - val_loss: 0.5508 - val_acc: 0.7139
Epoch 2/12
145056/145056 [==============================] - 108s - loss: 0.4651 - acc: 0.7799 - val_loss: 0.4537 - val_acc: 0.7832
Epoch 3/12
145056/145056 [==============================] - 124s - loss: 0.4421 - acc: 0.7942 - val_loss: 0.4561 - val_acc: 0.7875
Epoch 4/12
145056/145056 [==============================] - 128s - loss: 0.4256 - acc: 0.8033 - val_loss: 0.4361 - val_acc: 0.7999
Epoch 5/12
145056/145056 [==============================] - 131s - loss: 0.4124 - acc: 0.8118 - val_loss: 0.4292 - val_acc: 0.8045
Epoch 6/12
145056/145056 [==============================] - 114s - l

In [ ]:
#Model 2
np.random.seed(42)


epochs = 12
batch_size = 128


# Take a look at the shapes
print('X_train:', X_train.shape)
print('y_train:', y_train.shape)
print('X_test:', X_test.shape)
print('y_test:', y_test.shape)

tb_callback = keras.callbacks.TensorBoard(
        histogram_freq=0, write_graph=True)

model = build_model_twitter2()
model.fit(X_train, y_train,
          validation_data=(X_test, y_test),
          batch_size=batch_size, epochs=epochs,
          callbacks=[tb_callback])

In [ ]:
#Model 3
np.random.seed(42)


epochs = 12
batch_size = 128


# Take a look at the shapes
print('X_train:', X_train.shape)
print('y_train:', y_train.shape)
print('X_test:', X_test.shape)
print('y_test:', y_test.shape)

tb_callback = keras.callbacks.TensorBoard(
        histogram_freq=0, write_graph=True)

model = build_model_twitter3()
model.fit(X_train, y_train,
          validation_data=(X_test, y_test),
          batch_size=batch_size, epochs=epochs,
          callbacks=[tb_callback])

In [13]:
def build_model_twitter4(max_length=100,
                nb_filters=64,
                kernel_size=3,
                pool_size=2,
                regularization=0.01,
                weight_constraint=2.,
                dropout_prob=0.4,
                clear_session=True):
    if clear_session:
        K.clear_session()

    model = Sequential()
    model.add(Embedding(
        embeddings.shape[0],
        embeddings.shape[1],
        input_length=max_length,
        trainable=False,
        weights=[embeddings]))

    model.add(LSTM(100, recurrent_dropout = 0.2, dropout = 0.2))
    model.add(Dense(1, activation = 'sigmoid'))

    model.compile(
        loss='binary_crossentropy',
        optimizer='adam',
        metrics=['accuracy'])

    return model
    
def build_model_twitter5(max_length=100,
                filters=32,
                kernel_size=3,
                clear_session=True):
    if clear_session:
        K.clear_session()

    model = Sequential()
    model.add(Embedding(
        embeddings.shape[0],
        embeddings.shape[1],
        input_length=max_length,
        trainable=False,
        weights=[embeddings]))

    model.add(Conv1D(padding = "same", kernel_size = kernel_size, filters = filters, activation = "relu"))
    model.add(MaxPooling1D(pool_size = 2))
    model.add(LSTM(100))
    model.add(Dense(1, activation = 'sigmoid'))

    model.compile(
        loss='binary_crossentropy',
        optimizer='adam',
        metrics=['accuracy'])

    return model

In [ ]:
#Model 4
np.random.seed(42)


epochs = 12
batch_size = 128


# Take a look at the shapes
print('X_train:', X_train.shape)
print('y_train:', y_train.shape)
print('X_test:', X_test.shape)
print('y_test:', y_test.shape)

tb_callback = keras.callbacks.TensorBoard(
        histogram_freq=0, write_graph=True)

model = build_model_twitter3()
model.fit(X_train, y_train,
          validation_data=(X_test, y_test),
          batch_size=batch_size, epochs=epochs,
          callbacks=[tb_callback])

In [ ]:
#Model 5
np.random.seed(42)


epochs = 12
batch_size = 160


# Take a look at the shapes
print('X_train:', X_train.shape)
print('y_train:', y_train.shape)
print('X_test:', X_test.shape)
print('y_test:', y_test.shape)

tb_callback = keras.callbacks.TensorBoard(
        histogram_freq=0, write_graph=True)

model = build_model_twitter5()
model.fit(X_train, y_train,
          validation_data=(X_test, y_test),
          batch_size=batch_size, epochs=epochs,
          callbacks=[tb_callback])

In [ ]:
3

In [ ]:
import numpy as np
import keras
import argparse

from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

#from data_utils import load as load_data
from model import *


keras_model = KerasClassifier(build_fn=build_model_twitter5, verbose=1)
param_grid = {
    'filters': [32, 64],
    'kernel_size': [2, 3],
    'epochs': [12, 20],
    'batch_size': [64, 128, 160]
}
grid = GridSearchCV(estimator=keras_model, param_grid=param_grid)
grid_result = grid.fit(X_train, y_train)

# Summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/12
96704/96704 [==============================] - 211s - loss: 0.6853 - acc: 0.5186   
Epoch 2/12
96704/96704 [==============================] - 200s - loss: 0.5500 - acc: 0.6848   
Epoch 3/12
96704/96704 [==============================] - 201s - loss: 0.4408 - acc: 0.7893   
Epoch 4/12
96704/96704 [==============================] - 192s - loss: 0.4174 - acc: 0.8033   
Epoch 5/12
96704/96704 [==============================] - 217s - loss: 0.4011 - acc: 0.8128   
Epoch 6/12
96704/96704 [==============================] - 216s - loss: 0.3892 - acc: 0.8200   
Epoch 7/12
96704/96704 [==============================] - 256s - loss: 0.3777 - acc: 0.8279   
Epoch 8/12
96704/96704 [==============================] - 234s - loss: 0.3696 - acc: 0.8318   
Epoch 9/12
96704/96704 [==============================] - 185s - loss: 0.3582 - acc: 0.8378   
Epoch 10/12
96704/96704 [==============================] - 180s - loss: 0.3496 - acc: 0.8421   
Epoch 11/12
96704/96704 [========================

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



96704/96704 [==============================] - 183s - loss: 0.6939 - acc: 0.5024   
Epoch 3/12
96704/96704 [==============================] - 192s - loss: 0.6940 - acc: 0.4996   
Epoch 4/12
82432/96704 [========================>.....] - ETA: 28s - loss: 0.6936 - acc: 0.5057

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



96704/96704 [==============================] - 193s - loss: 0.4456 - acc: 0.7864   
Epoch 7/12
84864/96704 [=========================>....] - ETA: 22s - loss: 0.4144 - acc: 0.8055

...

96704/96704 [==============================] - 86s - loss: 0.4260 - acc: 0.7987    
Epoch 4/12
96704/96704 [==============================] - 88s - loss: 0.4070 - acc: 0.8094    
Epoch 5/12
96704/96704 [==============================] - 88s - loss: 0.3915 - acc: 0.8180    
Epoch 6/12
96704/96704 [==============================] - 87s - loss: 0.3783 - acc: 0.8260    
Epoch 7/12
96704/96704 [==============================] - 86s - loss: 0.3660 - acc: 0.8336    
Epoch 8/12
64480/96704 [===================>..........] - ETA: 28s - loss: 0.3519 - acc: 0.8396

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



96704/96704 [==============================] - 87s - loss: 0.4134 - acc: 0.8070    
Epoch 4/12
96704/96704 [==============================] - 87s - loss: 0.3977 - acc: 0.8165    
Epoch 5/12
96704/96704 [==============================] - 88s - loss: 0.3829 - acc: 0.8248    
Epoch 6/12
96704/96704 [==============================] - 87s - loss: 0.3701 - acc: 0.8326    
Epoch 7/12
96704/96704 [==============================] - 87s - loss: 0.3580 - acc: 0.8394    
Epoch 8/12
96704/96704 [==============================] - 87s - loss: 0.3483 - acc: 0.8459    
Epoch 9/12
96704/96704 [==============================] - 87s - loss: 0.3361 - acc: 0.8515    
Epoch 10/12
96704/96704 [==============================] - 87s - loss: 0.3280 - acc: 0.8558    
Epoch 11/12
96704/96704 [==============================] - 87s - loss: 0.3217 - acc: 0.8598    
Epoch 12/12
96704/96704 [==============================] - 89s - loss: 0.6284 - acc: 0.6119    
Epoch 2/12
96704/96704 [==============================] - 

96704/96704 [==============================] - 84s - loss: 0.5148 - acc: 0.7298    
Epoch 2/20
96704/96704 [==============================] - 83s - loss: 0.4344 - acc: 0.7939    
Epoch 3/20
96704/96704 [==============================] - 83s - loss: 0.4137 - acc: 0.8080    
Epoch 4/20
96704/96704 [==============================] - 84s - loss: 0.4001 - acc: 0.8148    
Epoch 5/20
96704/96704 [==============================] - 83s - loss: 0.3872 - acc: 0.8234    
Epoch 6/20
96704/96704 [==============================] - 84s - loss: 0.3808 - acc: 0.8260    
Epoch 7/20
96704/96704 [==============================] - 85s - loss: 0.3704 - acc: 0.8326    
Epoch 8/20
96704/96704 [==============================] - 85s - loss: 0.3642 - acc: 0.8359    
Epoch 9/20
96704/96704 [==============================] - 86s - loss: 0.3570 - acc: 0.8407    
Epoch 10/20
96704/96704 [==============================] - 86s - loss: 0.3494 - acc: 0.8442    
Epoch 11/20
96704/96704 [==============================] - 8

96704/96704 [==============================] - 89s - loss: 0.6937 - acc: 0.4978    
Epoch 5/20
96704/96704 [==============================] - 89s - loss: 0.6938 - acc: 0.4995    
Epoch 6/20
96704/96704 [==============================] - 88s - loss: 0.6936 - acc: 0.5000    
Epoch 7/20
96704/96704 [==============================] - 88s - loss: 0.6934 - acc: 0.5015    
Epoch 8/20
96704/96704 [==============================] - 89s - loss: 0.6935 - acc: 0.5007    
Epoch 9/20
96704/96704 [==============================] - 89s - loss: 0.6206 - acc: 0.6094    
Epoch 10/20
96704/96704 [==============================] - 89s - loss: 0.4512 - acc: 0.7843    
Epoch 11/20
96704/96704 [==============================] - 88s - loss: 0.4198 - acc: 0.8022    
Epoch 12/20
96704/96704 [==============================] - 88s - loss: 0.4013 - acc: 0.8133    
Epoch 13/20
96704/96704 [==============================] - 88s - loss: 0.3835 - acc: 0.8232    
Epoch 14/20
96704/96704 [==============================] 

In [7]:
#def marco_model():
#     model = Sequential()
#model.add(Embedding(max_features+1, size2, input_length=X_train.shape[1]))
#model.add(Embedding(embeddings_spacy.shape[0], embeddings_spacy.shape[1], input_length=features_train.shape[1]))
##model.add(GlobalAveragePooling1D())
#m#odel.add(Dense(1, activation='sigmoid'))
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

def marco_model(max_length=100,
                nb_filters=64,
                kernel_size=3,
                pool_size=2,
                regularization=0.01,
                weight_constraint=2.,
                dropout_prob=0.4,
                clear_session=True):
    if clear_session:
        K.clear_session()

    model = Sequential()
    model.add(Embedding(
        embeddings.shape[0],
        embeddings.shape[1],
        input_length=max_length,
        trainable=False,
        weights=[embeddings]))
    
    model.add(GlobalAveragePooling1D())
    model.add(Dense(1, activation='sigmoid'))

    #model.add(LSTM(100, recurrent_dropout = 0.2, dropout = 0.2))
    #model.add(Dense(1, activation = 'sigmoid'))

    model.compile(
        loss='binary_crossentropy',
        optimizer='adam',
        metrics=['accuracy'])

    return model

In [ ]:
np.random.seed(42)


epochs = 300
batch_size = 160


# Take a look at the shapes
print('X_train:', X_train.shape)
print('y_train:', y_train.shape)
print('X_test:', X_test.shape)
print('y_test:', y_test.shape)

tb_callback = keras.callbacks.TensorBoard(
        histogram_freq=0, write_graph=True)

model = marco_model()
model.fit(X_train, y_train,
          validation_data=(X_test, y_test),
          batch_size=batch_size, epochs=epochs,
          callbacks=[tb_callback])

X_train: (145056, 100)
y_train: (145056,)
X_test: (36265, 100)
y_test: (36265,)
Train on 145056 samples, validate on 36265 samples
Epoch 1/300
145056/145056 [==============================] - 9s - loss: 0.6600 - acc: 0.6346 - val_loss: 0.6358 - val_acc: 0.6671
Epoch 2/300
145056/145056 [==============================] - 9s - loss: 0.6203 - acc: 0.6731 - val_loss: 0.6077 - val_acc: 0.6840
Epoch 3/300
145056/145056 [==============================] - 9s - loss: 0.5982 - acc: 0.6876 - val_loss: 0.5902 - val_acc: 0.6947
Epoch 4/300
145056/145056 [==============================] - 8s - loss: 0.5836 - acc: 0.6970 - val_loss: 0.5779 - val_acc: 0.7026
Epoch 5/300
145056/145056 [==============================] - 9s - loss: 0.5729 - acc: 0.7034 - val_loss: 0.5685 - val_acc: 0.7079
Epoch 6/300
145056/145056 [==============================] - 9s - loss: 0.5646 - acc: 0.7087 - val_loss: 0.5612 - val_acc: 0.7131
Epoch 7/300
145056/145056 [==============================] - 9s - loss: 0.5579 - acc: 0.7

145056/145056 [==============================] - 6s - loss: 0.5037 - acc: 0.7509 - val_loss: 0.5060 - val_acc: 0.7505
Epoch 63/300
145056/145056 [==============================] - 6s - loss: 0.5035 - acc: 0.7509 - val_loss: 0.5059 - val_acc: 0.7503
Epoch 64/300
145056/145056 [==============================] - 6s - loss: 0.5034 - acc: 0.7509 - val_loss: 0.5058 - val_acc: 0.7505
Epoch 65/300
145056/145056 [==============================] - 6s - loss: 0.5032 - acc: 0.7509 - val_loss: 0.5058 - val_acc: 0.7511
Epoch 66/300
145056/145056 [==============================] - 6s - loss: 0.5031 - acc: 0.7512 - val_loss: 0.5055 - val_acc: 0.7511
Epoch 67/300
145056/145056 [==============================] - 6s - loss: 0.5030 - acc: 0.7511 - val_loss: 0.5054 - val_acc: 0.7506
Epoch 68/300
145056/145056 [==============================] - 6s - loss: 0.5028 - acc: 0.7514 - val_loss: 0.5055 - val_acc: 0.7514
Epoch 69/300
145056/145056 [==============================] - 6s - loss: 0.5027 - acc: 0.7513 - 

145056/145056 [==============================] - 6s - loss: 0.4989 - acc: 0.7543 - val_loss: 0.5020 - val_acc: 0.7527
Epoch 125/300
145056/145056 [==============================] - 6s - loss: 0.4989 - acc: 0.7540 - val_loss: 0.5019 - val_acc: 0.7531
Epoch 126/300
145056/145056 [==============================] - 6s - loss: 0.4988 - acc: 0.7540 - val_loss: 0.5019 - val_acc: 0.7530
Epoch 127/300
145056/145056 [==============================] - 6s - loss: 0.4988 - acc: 0.7544 - val_loss: 0.5018 - val_acc: 0.7532
Epoch 128/300
145056/145056 [==============================] - 6s - loss: 0.4987 - acc: 0.7543 - val_loss: 0.5018 - val_acc: 0.7530
Epoch 129/300
145056/145056 [==============================] - 6s - loss: 0.4987 - acc: 0.7543 - val_loss: 0.5019 - val_acc: 0.7531
Epoch 130/300
145056/145056 [==============================] - 6s - loss: 0.4987 - acc: 0.7542 - val_loss: 0.5018 - val_acc: 0.7531
Epoch 131/300
145056/145056 [==============================] - 6s - loss: 0.4986 - acc: 0.

145056/145056 [==============================] - 6s - loss: 0.4974 - acc: 0.7553 - val_loss: 0.5007 - val_acc: 0.7537
Epoch 187/300
145056/145056 [==============================] - 6s - loss: 0.4974 - acc: 0.7554 - val_loss: 0.5007 - val_acc: 0.7538
Epoch 188/300
145056/145056 [==============================] - 6s - loss: 0.4974 - acc: 0.7555 - val_loss: 0.5009 - val_acc: 0.7554
Epoch 189/300
145056/145056 [==============================] - 6s - loss: 0.4974 - acc: 0.7556 - val_loss: 0.5007 - val_acc: 0.7538
Epoch 190/300
145056/145056 [==============================] - 6s - loss: 0.4974 - acc: 0.7554 - val_loss: 0.5007 - val_acc: 0.7535
Epoch 191/300
145056/145056 [==============================] - 6s - loss: 0.4974 - acc: 0.7554 - val_loss: 0.5007 - val_acc: 0.7542
Epoch 192/300
145056/145056 [==============================] - 6s - loss: 0.4973 - acc: 0.7552 - val_loss: 0.5006 - val_acc: 0.7538
Epoch 193/300
145056/145056 [==============================] - 6s - loss: 0.4973 - acc: 0.

145056/145056 [==============================] - 5s - loss: 0.4968 - acc: 0.7565 - val_loss: 0.5003 - val_acc: 0.7547
Epoch 249/300
145056/145056 [==============================] - 5s - loss: 0.4968 - acc: 0.7563 - val_loss: 0.5003 - val_acc: 0.7546
Epoch 250/300
145056/145056 [==============================] - 6s - loss: 0.4968 - acc: 0.7563 - val_loss: 0.5002 - val_acc: 0.7546
Epoch 251/300
145056/145056 [==============================] - 5s - loss: 0.4967 - acc: 0.7562 - val_loss: 0.5002 - val_acc: 0.7548
Epoch 252/300
145056/145056 [==============================] - 6s - loss: 0.4968 - acc: 0.7559 - val_loss: 0.5004 - val_acc: 0.7551
Epoch 253/300
145056/145056 [==============================] - 6s - loss: 0.4967 - acc: 0.7563 - val_loss: 0.5002 - val_acc: 0.7548
Epoch 254/300
145056/145056 [==============================] - 6s - loss: 0.4967 - acc: 0.7564 - val_loss: 0.5002 - val_acc: 0.7548
Epoch 255/300
145056/145056 [==============================] - 5s - loss: 0.4967 - acc: 0.